### MFA + sampling

In [1]:
import pandas as pd
import numpy as np
import time
import ast
import sys
import escher
from BFAIR.INCA import INCA_reimport

Academic license - for non-commercial use only - expires 2021-05-28
Using license file /Users/matmat/gurobi.lic


Determination of memory status is not supported on this 
 platform, measuring for memoryleaks will never fail


#### INCA re-import

In [2]:
%pwd

'/Users/matmat/Documents/GitHub/AutoFlow-OmicsDataHandling/examples'

In [3]:
%cd /Users/matmat/Documents/Random data/Stefano
filename = 'data/MFA_modelInputsData/TestFile.mat'

/Users/matmat/Documents/Random data/Stefano


In [4]:
%cd /Users/matmat/Documents/GitHub/AutoFlow-OmicsDataHandling/examples

/Users/matmat/Documents/GitHub/AutoFlow-OmicsDataHandling/examples


In [5]:
#filename = 'data/MFA_modelInputsData/TestFile.mat'
simulation_info = pd.read_csv('data/MFA_modelInputsData/Re-import/experimentalMS_data_I.csv')
simulation_id = 'WTEColi_113C80_U13C20_01'

In [6]:
reimport_data = INCA_reimport()

In [7]:
fittedData, fittedFluxes, fittedFragments, fittedMeasuredFluxes, fittedMeasuredFragments, fittedMeasuredFluxResiduals, fittedMeasuredFragmentResiduals, simulationParameters = reimport_data.reimport(filename, simulation_info, simulation_id)

In [8]:
MFA_fluxes = {}
for i, rxn in fittedFluxes.iterrows():
    MFA_fluxes[rxn['rxn_id']] = [rxn['flux_ub'], rxn['flux_lb']]

In [79]:
from cobra.io import load_json_model
import pandas as pd
from math import floor
import csv
import cobra.test
from cobra.sampling import sample
from cobra.flux_analysis import single_gene_deletion, pfba
from cobra.flux_analysis import moma

In [80]:
#s = sample(model, 100)
#s.head()

In [81]:
model = load_json_model('data/FIA_MS_example/database_files/iJO1366.json')
model.solver = 'glpk'
solution = model.optimize()

In [82]:
#with model:
cnt = 0
problems = []
for rxn, bounds in MFA_fluxes.items():
    #print(cnt)
    #if cnt == 15:
    #    break
    if cnt == 7 or cnt == 72:
        cnt += 1
        continue
    #print(rxn, bounds)
    cnt += 1
    if bounds[0] < bounds[1]: # to fix the issue with negaive values above
        try:
            model.reactions.get_by_id(rxn).lower_bound = round(bounds[0], 1)
            model.reactions.get_by_id(rxn).upper_bound = round(bounds[1], 1)
        except KeyError:
            continue
            print(f'Did not work for', rxn)
    else:
        try:
            model.reactions.get_by_id(rxn).upper_bound = round(bounds[0], 1)
            model.reactions.get_by_id(rxn).lower_bound = round(bounds[1], 1)
        except KeyError:
            continue
            print(f'Did not work for', rxn)
    solution_after2 = model.optimize() #pfba(model)
    if solution_after2.objective_value > 0:
        print(f'Solution after adding {rxn}: {solution_after2.objective_value:.2f}')
    else:
        print(f'Solution infeasible if adding {rxn}, count is {cnt-1}')
        problems = problems.append(rnx)
#solution_after=pfba(model)
%time sampled_fluxes_after2 = sample(model, 100, processes=2)

Solution after adding AKGDH: 0.98
Solution after adding ALATA_L: 0.98
Solution after adding ASNN: 0.98
Solution after adding ATPM: 0.99
Solution after adding CS: 0.99
Solution after adding DAPDC: 0.58
Solution after adding EDA: 0.58
Solution after adding EDD: 0.58
Solution after adding FBA: 0.58
Solution after adding FUM: 0.58
Solution after adding GHMT2r: 0.58
Solution after adding GLCptspp: 0.58
Solution after adding GLCt2pp: 0.58
Solution after adding GLNS: 0.28
Solution after adding GND: 0.28
Solution after adding HEX1: 0.28
Solution after adding ICDHyr: 0.28
Solution after adding ICL: 0.28
Solution after adding MALS: 0.28
Solution after adding MDH: 0.28
Solution after adding ME1: 0.28
Solution after adding ME2: 0.28
Solution after adding MTHFC: 0.28
Solution after adding MTHFD: 0.28
Solution after adding PDH: 0.28
Solution after adding PFK: 0.28
Solution after adding PGI: 0.28
Solution after adding PGM: 0.28
Solution after adding PPC: 0.28
Solution after adding PPCK: 0.28
Solution

In [84]:
problems

[]

In [14]:
print('Before adding new constraints:', solution)
print('After adding new constraints:', solution_after2)
print('After adding new constraints + pFBA:', solution_after)

Before adding new constraints: <Solution 0.982 at 0x7fb9c22702e0>
After adding new constraints: <Solution -0.000 at 0x7fb9c1251040>
After adding new constraints + pFBA: <Solution 193.259 at 0x7fb9c1251160>


In [15]:
moma_after_MFA = moma(model=model, solution=solution, linear=True)

In [16]:
import escher

In [59]:
#fluxes = {}
#for cnt, col in enumerate(sampled_fluxes.columns):
#    fluxes[col] = sampled_fluxes.values[0][cnt]
fluxes_after = {}
for cnt, col in enumerate(sampled_fluxes_after2.columns):
    fluxes_after[col] = sampled_fluxes_after2.values[0][cnt]
#fluxes_moma = {}
#reactions = list(moma_after_MFA.fluxes.index)
#for cnt, row in enumerate(moma_after_MFA.fluxes):
#    fluxes_moma[reactions[cnt]] = row

In [60]:
b = escher.Builder('e_coli_core.Core metabolism',
               reaction_data= fluxes_after).display_in_notebook()

In [61]:
b

In [ ]:
reactions_to_relax
metabolites_to_relax

In [73]:
from gurobipy import Model as GRBModel

In [74]:
from BFAIR import thermo, thermo_models

In [75]:
model.solver = 'optlang-gurobi'

In [76]:
if model.solver.interface.__name__ != "optlang.gurobi_interface":
    raise ModuleNotFoundError("Requires Gurobi.")

In [77]:
# copy Gurobi model
grb_model: GRBModel = model.solver.problem.copy()

In [106]:
list(MFA_fluxes.keys())

['26dap_DASH_MSYN',
 'ACONTa_ACONTb',
 'ACONTa_ACONTb_reverse',
 'AKGDH',
 'ALATA_L',
 'ArgSYN',
 'ASNN',
 'ASPTA',
 'ATPM',
 'CS',
 'DAPDC',
 'Ec_Biomass_INCA',
 'EDA',
 'EDD',
 'EX_ac_LPAREN_e_RPAREN_',
 'EX_co2_LPAREN_e_RPAREN_',
 'EX_co2_LPAREN_e_RPAREN__unlabeled',
 'EX_glc_LPAREN_e_RPAREN_',
 'EX_nh4_LPAREN_e_RPAREN_',
 'EX_o2_LPAREN_e_RPAREN_',
 'EX_so4_LPAREN_e_RPAREN_',
 'FADR_NADH_CYTBD_HYD_ATPS4r',
 'FBA',
 'FBA_reverse',
 'FUM',
 'FUM_reverse',
 'G6PDH2r_PGL',
 'GAPD_PGK',
 'GAPD_PGK_reverse',
 'GHMT2r',
 'GHMT2r_reverse',
 'GLCptspp',
 'GLCt2pp',
 'GLNS',
 'GluSYN',
 'GND',
 'HEX1',
 'HisSYN',
 'ICDHyr',
 'ICDHyr_reverse',
 'ICL',
 'IleSYN',
 'LeuSYN',
 'MALS',
 'MDH',
 'MDH_reverse',
 'ME1',
 'ME2',
 'MetSYN',
 'MlthfSYN',
 'MlthfSYN_reverse',
 'MTHFC',
 'MTHFD',
 'NADH_CYTBD_HYD_ATPS4r',
 'NADTRHD_THD2pp',
 'NADTRHD_THD2pp_reverse',
 'PDH',
 'PFK',
 'PGI',
 'PGI_reverse',
 'PGM',
 'PGM_reverse',
 'PheSYN',
 'PPC',
 'PPCK',
 'ProSYN',
 'PRPPS',
 'PTAr_ACKr_ACS',
 'PTAr_AC

In [78]:
reactions_to_relax = list(MFA_fluxes.keys())

In [33]:
grb_model.getVars()

[<gurobi.Var EX_cm_e>,
 <gurobi.Var EX_cm_e_reverse_d60e9>,
 <gurobi.Var EX_cmp_e>,
 <gurobi.Var EX_cmp_e_reverse_e7a73>,
 <gurobi.Var EX_co2_e>,
 <gurobi.Var EX_co2_e_reverse_d0466>,
 <gurobi.Var EX_cobalt2_e>,
 <gurobi.Var EX_cobalt2_e_reverse_2bf0e>,
 <gurobi.Var DM_4crsol_c>,
 <gurobi.Var DM_4crsol_c_reverse_c57fa>,
 <gurobi.Var DM_5drib_c>,
 <gurobi.Var DM_5drib_c_reverse_37606>,
 <gurobi.Var DM_aacald_c>,
 <gurobi.Var DM_aacald_c_reverse_6df50>,
 <gurobi.Var DM_amob_c>,
 <gurobi.Var DM_amob_c_reverse_90c8f>,
 <gurobi.Var DM_mththf_c>,
 <gurobi.Var DM_mththf_c_reverse_46bf0>,
 <gurobi.Var EX_colipa_e>,
 <gurobi.Var EX_colipa_e_reverse_f9125>,
 <gurobi.Var DM_oxam_c>,
 <gurobi.Var DM_oxam_c_reverse_55504>,
 <gurobi.Var EX_glc__D_e>,
 <gurobi.Var EX_glc__D_e_reverse_af641>,
 <gurobi.Var EX_glcn_e>,
 <gurobi.Var EX_glcn_e_reverse_9e36b>,
 <gurobi.Var BIOMASS_Ec_iJO1366_WT_53p95M>,
 <gurobi.Var BIOMASS_Ec_iJO1366_WT_53p95M_reverse_06c4a>,
 <gurobi.Var EX_glcr_e>,
 <gurobi.Var EX_glcr_

In [119]:
#metabolites_to_relax = []
#for rxn in model.reactions:
#    if rxn.id in list(MFA_fluxes.keys()):
#        metabolites_to_relax += [react.id for react in rxn.reactants]
#metabolites_to_relax = list(set(metabolites_to_relax))

In [32]:
#grb_model.getConstrs()

[<gurobi.Constr 3amp_p>,
 <gurobi.Constr anhgm4p_c>,
 <gurobi.Constr mobd_e>,
 <gurobi.Constr met__L_e>,
 <gurobi.Constr halipa_e>,
 <gurobi.Constr 3oddecACP_c>,
 <gurobi.Constr lgt__S_c>,
 <gurobi.Constr lac__D_e>,
 <gurobi.Constr isetac_c>,
 <gurobi.Constr h2_p>,
 <gurobi.Constr xdp_c>,
 <gurobi.Constr amob_c>,
 <gurobi.Constr tmao_c>,
 <gurobi.Constr 3amac_c>,
 <gurobi.Constr ttdcea_c>,
 <gurobi.Constr galctn__L_p>,
 <gurobi.Constr ac_p>,
 <gurobi.Constr uaagmda_c>,
 <gurobi.Constr prbatp_c>,
 <gurobi.Constr anhgm3p_p>,
 <gurobi.Constr arbtn_fe3_c>,
 <gurobi.Constr ppap_c>,
 <gurobi.Constr 4per_c>,
 <gurobi.Constr k_c>,
 <gurobi.Constr gdp_e>,
 <gurobi.Constr phe__L_e>,
 <gurobi.Constr dc2coa_c>,
 <gurobi.Constr ala_B_e>,
 <gurobi.Constr 3ohexACP_c>,
 <gurobi.Constr gln__L_e>,
 <gurobi.Constr cddec5eACP_c>,
 <gurobi.Constr moco_c>,
 <gurobi.Constr 2agpe141_p>,
 <gurobi.Constr ipdp_c>,
 <gurobi.Constr appl_c>,
 <gurobi.Constr progly_p>,
 <gurobi.Constr dgsn_e>,
 <gurobi.Constr uacgam

In [79]:
metabolites_to_relax = []

In [30]:
model.reactions

[<Reaction EX_cm_e at 0x7ff65f3e48e0>,
 <Reaction EX_cmp_e at 0x7ff65f3e4940>,
 <Reaction EX_co2_e at 0x7ff65f3e49a0>,
 <Reaction EX_cobalt2_e at 0x7ff65f3e4a60>,
 <Reaction DM_4crsol_c at 0x7ff65f3e4b20>,
 <Reaction DM_5drib_c at 0x7ff65f3e4be0>,
 <Reaction DM_aacald_c at 0x7ff65f3e4ca0>,
 <Reaction DM_amob_c at 0x7ff65f3e4d60>,
 <Reaction DM_mththf_c at 0x7ff65f3e4e20>,
 <Reaction EX_colipa_e at 0x7ff65f3e4ee0>,
 <Reaction DM_oxam_c at 0x7ff65f3e4fa0>,
 <Reaction EX_glc__D_e at 0x7ff65f3e4970>,
 <Reaction EX_glcn_e at 0x7ff65f3e4910>,
 <Reaction BIOMASS_Ec_iJO1366_WT_53p95M at 0x7ff65f3fc220>,
 <Reaction EX_glcr_e at 0x7ff65f3fc2e0>,
 <Reaction EX_colipap_e at 0x7ff65f3fc0a0>,
 <Reaction EX_glcur_e at 0x7ff65f3fc3a0>,
 <Reaction EX_glcur1p_e at 0x7ff65f3fc160>,
 <Reaction BIOMASS_Ec_iJO1366_core_53p95M at 0x7ff65f432220>,
 <Reaction EX_12ppd__R_e at 0x7ff65f4322b0>,
 <Reaction EX_gln__L_e at 0x7ff65f432070>,
 <Reaction EX_cpgn_e at 0x7ff65f432370>,
 <Reaction EX_glu__L_e at 0x7ff65f4

In [80]:
model.reactions[0].flux_expression

1.0*EX_cm_e - 1.0*EX_cm_e_reverse_d60e9

In [81]:
model.reactions[6].reverse_id

'DM_aacald_c_reverse_6df50'

In [82]:
relax_vars_reverse

[<gurobi.Var AKGDH_reverse_08bdc (value 0.0)>,
 <gurobi.Var ALATA_L_reverse_e54ff (value 0.0)>,
 <gurobi.Var ASNN_reverse_515f8 (value 0.0)>,
 <gurobi.Var ASPTA_reverse_36525 (value 0.2)>,
 <gurobi.Var ATPM_reverse_5b752 (value 0.0)>,
 <gurobi.Var CS_reverse_8d7e9 (value 0.0)>,
 <gurobi.Var DAPDC_reverse_d3ab8 (value 0.0)>,
 <gurobi.Var EDA_reverse_81f1b (value 0.0)>,
 <gurobi.Var EDD_reverse_007a2 (value 0.0)>,
 <gurobi.Var FUM_reverse_d3642 (value 0.0)>,
 <gurobi.Var FBA_reverse_84806 (value 0.0)>,
 <gurobi.Var GHMT2r_reverse_d977f (value 0.0)>,
 <gurobi.Var GLCptspp_reverse_9cf76 (value 0.0)>,
 <gurobi.Var GLCt2pp_reverse_b9e3b (value 0.0)>,
 <gurobi.Var GLNS_reverse_59581 (value 0.0)>,
 <gurobi.Var GND_reverse_eec5c (value 0.0)>,
 <gurobi.Var HEX1_reverse_25efa (value 0.0)>,
 <gurobi.Var ICDHyr_reverse_7f84b (value 0.0)>,
 <gurobi.Var ICL_reverse_2f27e (value 0.0)>,
 <gurobi.Var MALS_reverse_d7382 (value 0.0)>,
 <gurobi.Var MDH_reverse_ee52c (value 0.0)>,
 <gurobi.Var ME1_reverse_9

In [24]:
relax_cons = [
    grb_model.getConstrByName(con.name) for con in model.metabolites if con.id in metabolites_to_relax
]
relax_vars = [
    grb_model.getVarByName(con.id) for con in model.reactions if con.id in reactions_to_relax
]
relax_vars_reverse = [
    grb_model.getVarByName(con.reverse_id) for con in model.reactions if con.id in reactions_to_relax
]

In [83]:
relax_cons = [
    grb_model.getConstrByName(con.name) for con in model.metabolites if con.id in metabolites_to_relax
]
relax_vars = [
    grb_model.getVarByName(con.id) for con in model.reactions if con.id in reactions_to_relax
]
relax_vars_reverse = [
    grb_model.getVarByName(con.reverse_id) for con in model.reactions if con.id in reactions_to_relax
]

relax_vars = relax_vars + relax_vars_reverse

cons_penalties = [1] * len(relax_cons)
vars_penalties = [1] * len(relax_vars)

# perform relaxation of variable bounds
relax_obj = grb_model.feasRelax(
    relaxobjtype=0,
    minrelax=True,
    vars=relax_vars,
    lbpen=vars_penalties,
    ubpen=vars_penalties,
    constrs=relax_cons,
    rhspen=cons_penalties,
)

In [50]:
relax_cons

[]

In [84]:
relax_obj

1.5

In [57]:
model.optimize()

,fluxes,reduced_costs
EX_cm_e,0.0,0.000000
EX_cmp_e,0.0,-2.754552
EX_co2_e,9.4,0.000000
EX_cobalt2_e,0.0,0.000000
DM_4crsol_c,0.0,0.000000
...,...,...
RNDR4,0.0,0.000000
RNDR4b,0.0,0.000000
RNTR1c2,0.0,0.000000
RNTR2c2,0.0,-2.754552


In [85]:
grb_model.optimize()

In [89]:
# check if relaxation was successful
if relax_obj < 0:
    raise Infeasible("Failed to create the feasibility relaxation!")

grb_model.optimize()

In [52]:
dir(relax_vars[0])

['BarX',
 'BranchPriority',
 'IISLB',
 'IISUB',
 'LB',
 'Obj',
 'ObjN',
 'PStart',
 'PWLObjCvx',
 'Partition',
 'PreFixVal',
 'RC',
 'SALBLow',
 'SALBUp',
 'SAObjLow',
 'SAObjUp',
 'SAUBLow',
 'SAUBUp',
 'ScenNLB',
 'ScenNObj',
 'ScenNUB',
 'ScenNX',
 'Start',
 'UB',
 'UnbdRay',
 'VBasis',
 'VDualNorm',
 'VTag',
 'VType',
 'VarHintPri',
 'VarHintVal',
 'VarName',
 'VarPreStat',
 'X',
 'Xn',
 '__add__',
 '__cindex__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__isub__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__numcols__',
 '__pow__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__rsub__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_cmodel',
 '_colno',


In [54]:
variables = [var.VarName for var in relax_vars]

In [90]:
grb_model

<gurobi.Model Continuous instance _copy: 1958 constrs, 5318 vars, Parameter changes: FeasibilityTol=1e-07, Method=0, Presolve=0, OutputFlag=0>

In [101]:
_extract_bound_violations(grb_model, variables, "ArtL_", "ArtU_")

[('ASPTA_reverse_36525', 0, 1.3000001),
 ('DAPDC_reverse_d3ab8', 0, 0.20000009999999996)]

In [123]:
[reaction for reaction in model.reactions if reaction.id == 'ASPTA'][0].reverse_variable.lb

0

In [125]:
model.reactions.ASPTA.reverse_variable.lb

0

In [124]:
[reaction for reaction in model.reactions if reaction.reverse_id == 'ASPTA_reverse_36525'][0]

Reaction identifier,ASPTA
Name,Aspartate transaminase
Memory address,0x07ff64815b8b0
Stoichiometry,akg_c + asp__L_c --> glu__L_c + oaa_c 2-Oxoglutarate + L-Aspartate --> L-Glutamate + Oxaloacetate
GPR,b0928
Lower bound,1.3
Upper bound,1.3


In [100]:
[var.VarName for var in grb_model.getVars() if "ArtU" in var.VarName]

['ArtU_AKGDH',
 'ArtU_AKGDH_reverse_08bdc',
 'ArtU_ALATA_L',
 'ArtU_ALATA_L_reverse_e54ff',
 'ArtU_ASNN',
 'ArtU_ASNN_reverse_515f8',
 'ArtU_ASPTA',
 'ArtU_ASPTA_reverse_36525',
 'ArtU_ATPM',
 'ArtU_ATPM_reverse_5b752',
 'ArtU_CS',
 'ArtU_CS_reverse_8d7e9',
 'ArtU_DAPDC',
 'ArtU_DAPDC_reverse_d3ab8',
 'ArtU_EDA',
 'ArtU_EDA_reverse_81f1b',
 'ArtU_EDD',
 'ArtU_EDD_reverse_007a2',
 'ArtU_FUM',
 'ArtU_FUM_reverse_d3642',
 'ArtU_FBA',
 'ArtU_FBA_reverse_84806',
 'ArtU_GHMT2r',
 'ArtU_GHMT2r_reverse_d977f',
 'ArtU_GLCptspp',
 'ArtU_GLCptspp_reverse_9cf76',
 'ArtU_GLCt2pp',
 'ArtU_GLCt2pp_reverse_b9e3b',
 'ArtU_GLNS',
 'ArtU_GLNS_reverse_59581',
 'ArtU_GND',
 'ArtU_GND_reverse_eec5c',
 'ArtU_HEX1',
 'ArtU_HEX1_reverse_25efa',
 'ArtU_ICDHyr',
 'ArtU_ICDHyr_reverse_7f84b',
 'ArtU_ICL',
 'ArtU_ICL_reverse_2f27e',
 'ArtU_MALS',
 'ArtU_MALS_reverse_d7382',
 'ArtU_MDH',
 'ArtU_MDH_reverse_ee52c',
 'ArtU_ME1',
 'ArtU_ME1_reverse_9736c',
 'ArtU_ME2',
 'ArtU_ME2_reverse_2b0a2',
 'ArtU_MTHFC',
 'ArtU_

In [42]:
def _extract_bound_violations(grb_model, variables, negative_slack_prefix, positive_slack_prefix):
    # Extracts the magnitude of the bound violations on the relaxed Gurobi model
    epsilon = grb_model.Params.FeasibilityTol

    violations = []

    for var in variables:
        lb_change = -grb_model.getVarByName(negative_slack_prefix + var).X
        ub_change = grb_model.getVarByName(positive_slack_prefix + var).X

        if lb_change < -epsilon:
            violations.append((var, lb_change - epsilon, 0))
        elif ub_change > epsilon:
            violations.append((var, 0, ub_change + epsilon))

    return violations

In [164]:
model.optim

AttributeError: 'Model' object has no attribute 'optim'

In [163]:
grb_model.getVars()

[<gurobi.Var EX_cm_e (value 0.0)>,
 <gurobi.Var EX_cm_e_reverse_d60e9 (value 0.0)>,
 <gurobi.Var EX_cmp_e (value 0.0)>,
 <gurobi.Var EX_cmp_e_reverse_e7a73 (value 0.0)>,
 <gurobi.Var EX_co2_e (value 19.675222635663193)>,
 <gurobi.Var EX_co2_e_reverse_d0466 (value 0.0)>,
 <gurobi.Var EX_cobalt2_e (value 0.0)>,
 <gurobi.Var EX_cobalt2_e_reverse_2bf0e (value 2.4559295318174544e-05)>,
 <gurobi.Var DM_4crsol_c (value 0.00021906891423811693)>,
 <gurobi.Var DM_4crsol_c_reverse_c57fa (value 0.0)>,
 <gurobi.Var DM_5drib_c (value 0.0002210336578635709)>,
 <gurobi.Var DM_5drib_c_reverse_37606 (value 0.0)>,
 <gurobi.Var DM_aacald_c (value 0.0)>,
 <gurobi.Var DM_aacald_c_reverse_6df50 (value 0.0)>,
 <gurobi.Var DM_amob_c (value 1.9647436254539633e-06)>,
 <gurobi.Var DM_amob_c_reverse_90c8f (value 0.0)>,
 <gurobi.Var DM_mththf_c (value 0.0004401025721016971)>,
 <gurobi.Var DM_mththf_c_reverse_46bf0 (value 0.0)>,
 <gurobi.Var EX_colipa_e (value 0.0)>,
 <gurobi.Var EX_colipa_e_reverse_f9125 (value 0.0

In [ ]:
model.reactions.get_by_id(rxn).lower_bound = round(bounds[0], 1)
model.reactions.get_by_id(rxn).upper_bound = round(bounds[1], 1)

In [133]:
model.reactions.get_by_id('ASPTA').subsystem

'Alanine and Aspartate Metabolism'

In [118]:
import re

In [121]:
re.match('.+?(?=_reverse_)', 'ASPTA_reverse_36525')[0]

'ASPTA'

In [141]:
destructive = True

In [142]:
# transfer/record the standard dG changes to the ThermoModel
if relax_vars:
    rows = []
    violations = _extract_bound_violations(grb_model, variables, "ArtL_", "ArtU_")
    for rxn_id, lb_change, ub_change in violations:
        if destructive:
            if '_reverse_' in rxn_id:
                rxn_id = re.match('.+?(?=_reverse_)', rxn_id)[0]
                lb_change = -ub_change
                ub_change = 0
            lb = model.reactions.get_by_id(rxn_id).lower_bound
            ub = model.reactions.get_by_id(rxn_id).upper_bound
            model.reactions.get_by_id(rxn_id).lower_bound = lb + lb_change
            model.reactions.get_by_id(rxn_id).upper_bound = ub + ub_change
        rows.append(
            {
                "reaction": rxn_id,
                "bound_change": lb_change + ub_change,
                "subsystem": model.reactions.get_by_id(rxn_id).subsystem,
            }
        )
    cons_table = pd.DataFrame.from_records(rows, index="reaction")
else:
    cons_table = None

In [143]:
model.optimize()

,fluxes,reduced_costs
EX_cm_e,0.000000e+00,0.0
EX_cmp_e,0.000000e+00,-0.0
EX_co2_e,6.994450e+00,0.0
EX_cobalt2_e,-5.185539e-12,-0.0
DM_4crsol_c,0.000000e+00,0.0
...,...,...
RNDR4,0.000000e+00,0.0
RNDR4b,0.000000e+00,0.0
RNTR1c2,0.000000e+00,0.0
RNTR2c2,0.000000e+00,0.0


In [145]:
model.reactions.get_by_id('DAPDC')

Reaction identifier,DAPDC
Name,Diaminopimelate decarboxylase
Memory address,0x07ff6483b2f10
Stoichiometry,"26dap__M_c + h_c <=> co2_c + lys__L_c Meso-2,6-Diaminoheptanedioate + H+ <=> CO2 CO2 + L-Lysine"
GPR,b2838
Lower bound,-9.999999994736442e-08
Upper bound,0.2


In [146]:
sampled_fluxes = sample(model, 100, processes=2)

Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpwaiolkek.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp3n_ckrzj.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


In [147]:
fluxes_after = {}
for cnt, col in enumerate(sampled_fluxes.columns):
    fluxes_after[col] = sampled_fluxes.values[0][cnt]

In [148]:
b = escher.Builder('e_coli_core.Core metabolism',
               reaction_data= fluxes_after).display_in_notebook()

In [149]:
b

In [ ]:
# transfer/record the log concentration changes to the ThermoModel
if relax_vars:
    rows = []
    violations = _extract_bound_violations(
        grb_model, tmodel.log_concentration.list_attr("id"), "ArtL_LC_", "ArtU_LC_"
    )
    for met_id, lb_change, ub_change in violations:
        if destructive:
            var = tmodel.log_concentration.get_by_id(met_id).variable
            var.set_bounds(lb=var.lb + lb_change, ub=var.lb + ub_change)
        rows.append(
            {
                "metabolite": met_id,
                "bound_change": lb_change + ub_change,
                "compartment": tmodel.compartments[tmodel.metabolites.get_by_id(met_id).compartment]["name"],
            }
        )
    vars_table = pd.DataFrame.from_records(rows, index="metabolite")
else:
    vars_table = None

In [ ]:
# solve relaxed model
if destructive:
    tmodel.optimize()

In [38]:
return cons_table, vars_table

NameError: name 'tmodel' is not defined

In [35]:
class Timer:
    def __init__(self):
        return None
    
    def __enter__(self):
        print("---  start ---")
        self.start_time = time.time()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print("--- Run time: %.2f seconds ---" % (time.time() - self.start_time))
    
    def measure(self, function):
        function

In [37]:
print('a')

a


In [38]:
with Timer() as timer:
    timer.measure(print('a'))

---  start ---
a
--- Run time: 0.00 seconds ---


In [66]:
from contextlib import contextmanager

@contextmanager
def timer(function):
    try:
        print("---  start ---")
        start_time = time.time()
        yield function
    finally:
        print("--- Run time: %.2f seconds ---" % (time.time() - start_time))

In [68]:
with timer(print) as timed_printer:
    timed_printer('a')

---  start ---
a
--- Run time: 0.00 seconds ---


In [22]:
import time
import functools

In [23]:
def timer(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print("--- start ---")
        start_time = time.time()
        func(*args, **kwargs)
        print(f'{func.__name__} takes {(time.time() - start_time):.2f} seconds to run')
        print("--- end ---")
    return wrapper

In [26]:
@timer
def timed_printer(string: str):
    """Random Doc"""
    print(string)

In [27]:
timed_printer('a')

--- start ---
a
timed_printer takes 0.00 seconds to run
--- end ---


In [28]:
timed_printer.__doc__

'Random Doc'